### 自定义模型

https://tensorflow.google.cn/beta/guide/keras/custom_layers_and_models

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-rc1'

#### 1.多层感知机

In [2]:
class MLP(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.layer1 = tf.keras.layers.Dense(512,activation="relu")
        self.layer2 = tf.keras.layers.Dense(10)
    
    def call(self,inputs):
        return self.layer2(self.layer1(inputs))

In [6]:
mlp = MLP()
inputs = tf.random.normal(shape=(5,3))
mlp(inputs)

<tf.Tensor: id=125, shape=(5, 10), dtype=float32, numpy=
array([[ 7.83876888e-03, -1.08052760e-01, -1.20274723e-04,
        -1.80707231e-01,  5.25910035e-03, -4.92951460e-02,
         1.01656362e-01, -1.58705562e-02,  4.82824370e-02,
        -1.37929916e-01],
       [ 2.47069616e-02,  8.43866467e-02,  8.75827670e-03,
        -1.70194581e-02,  3.20906378e-02, -4.62367237e-02,
         1.12848513e-01,  2.22287886e-03, -2.74914317e-03,
        -6.15079328e-03],
       [ 1.14922315e-01,  2.92703420e-01, -1.24090314e-02,
        -6.07170984e-02,  2.36929953e-01, -2.23907381e-01,
         2.77480423e-01, -2.48807669e-02,  2.57573761e-02,
        -3.62296402e-02],
       [ 1.14844553e-03, -2.24759802e-02, -1.77966896e-03,
        -1.01343520e-01,  7.89665990e-03, -7.01378062e-02,
         8.25694054e-02,  2.48313770e-02, -3.71037796e-03,
        -1.64541513e-01],
       [-5.59377000e-02, -7.30469897e-02, -2.59499010e-02,
        -2.03245297e-01, -9.33973864e-02, -1.56286452e-02,
         8.23

### 2.序贯模型

In [7]:
class MySequentail(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(MySequentail,self).__init__(**kwargs)
        self._layers = []
    def add(self,layer):
        self._layers.append(layer)
    def call(self,x):
        for i in range(len(self._layers)):
            x = self._layers[i](x)
        return x

In [8]:
model = MySequentail()
x = tf.random.normal(shape=(3,3))
model.add(tf.keras.layers.Dense(8,activation="relu"))
model.add(tf.keras.layers.Dense(4,activation="relu"))
model.add(tf.keras.layers.Dense(2,activation="relu"))
model(x)

<tf.Tensor: id=212, shape=(3, 2), dtype=float32, numpy=
array([[0.        , 1.5620372 ],
       [0.5701231 , 0.27675113],
       [0.        , 1.2067871 ]], dtype=float32)>

#### 3.复杂的模型
    使用流程控制

In [28]:
@tf.function
def fancy(x):
    print(tf.norm(x))
    while tf.norm(x).numpy()>1:
        x /= 2
    
    if tf.norm(x).numpy()<0.8:
        x *= 10
    
    return tf.reduce_sum(x)

In [30]:
tf.norm(x)

<tf.Tensor: id=319, shape=(), dtype=float32, numpy=2.6849563>

In [35]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x

In [37]:
square_if_positive(2)

<tf.Tensor: id=352, shape=(), dtype=int32, numpy=4>

In [66]:
class FancyMLP(tf.keras.layers.Layer):
    def __init__(self,units):
        super(FancyMLP,self).__init__()
        w_init = tf.keras.initializers.he_normal()
        b_init = tf.keras.initializers.ones()
#         self.weight = tf.Variable(initial_value = w_init(shape = (units,units),dtype = "float32"))
        self.weight = self.add_weight(shape = (units,units),initializer=w_init)
        self.bais = tf.Variable(initial_value = b_init(shape = (units,),dtype = "float32"))
        self.dense = tf.keras.layers.Dense(units=units)
    
    def fancy(x):
        print(x)
        while tf.norm(x).numpy()>1:
            x /= 2

        if tf.norm(x).numpy()<0.8:
            x *= 10

        return tf.reduce_sum(x)
    
    def call(self,x):
        x = self.dense(x)
        x = tf.keras.activations.relu(tf.matmul(x,self.weight) + self.bais)
        x = self.dense(x)
        print(x)
        while tf.norm(x).numpy()>1:
            x /= 2
        if tf.norm(x).numpy()<0.8:
            x *= 10
        return tf.reduce_sum(x)

In [67]:
x2 = tf.random.uniform(shape=(2,20))

In [68]:
fancyMlp = FancyMLP(units=20)
fancyMlp(x2)

tf.Tensor(
[[-2.340381    1.0914574   0.17550653 -0.9373035  -1.4193647  -1.4479309
   0.8340757   0.9973731   1.8684517   0.7114239   0.8644439   0.2535656
   1.2517823  -0.55293125 -0.2708936  -0.81915826 -0.6796769   0.11186872
  -0.5061555  -0.7511518 ]
 [-2.8369555   1.1116349   0.7797055  -0.84308916 -1.721367   -1.8832062
   0.26153195  1.4780743   1.6890855  -0.11013949 -0.2908042   0.28961733
   1.7603047  -1.320871    0.02914088 -0.45108068 -0.46857625 -1.0595913
  -0.5052365  -0.8992709 ]], shape=(2, 20), dtype=float32)


<tf.Tensor: id=1101, shape=(), dtype=float32, numpy=-0.8195115>

#### 4.联合调用

In [61]:
class NestMLP(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        
        self.net = MySequentail()
        self.net.add(tf.keras.layers.Dense(64,activation="relu"))
        self.net.add(tf.keras.layers.Dense(32,activation="relu"))
        self.dense = tf.keras.layers.Dense(16,activation="relu")
    def call(self,x):
        return self.dense(self.net(x))

In [62]:
nestMLP = NestMLP()
nestMLP(x)

<tf.Tensor: id=918, shape=(3, 16), dtype=float32, numpy=
array([[0.        , 0.        , 0.31013465, 0.315785  , 0.        ,
        0.33354864, 0.        , 0.2869964 , 0.24004821, 0.07661889,
        0.16816887, 0.16114146, 0.07677162, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.4415934 , 0.10768186, 0.0395275 ,
        0.        , 0.        , 0.04496279, 0.25970137, 0.        ,
        0.01802109, 0.13638093, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.18038486, 0.29828128, 0.        ,
        0.3663038 , 0.        , 0.0944141 , 0.19359283, 0.05123146,
        0.22565955, 0.0198496 , 0.21072379, 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [64]:
model = tf.keras.Sequential()
model.add(nestMLP)
model.add(tf.keras.layers.Dense(20))
model.add(fancyMlp)

In [65]:
model(x)

tf.Tensor(
[[ 0.38004476  0.7346415  -1.8990432  -0.3343426   0.73921734 -0.6828878
   0.67070854 -0.9022436  -0.1926144  -0.47618344 -1.5466332   0.07314962
   0.9879817  -0.37642935  0.3311301   0.7130318  -1.0437785   0.11673608
   1.9734795  -1.4942398 ]
 [ 0.45384002  0.7804333  -1.8151624   0.15711592  0.5680583  -0.5109081
   0.64784896 -0.9032785   0.03367949 -0.63627976 -1.6567736   0.14908725
   1.0143178  -0.14619279  0.48874995  0.756037   -1.099119    0.25569266
   2.055519   -1.5298482 ]
 [ 0.3626921   0.65739745 -1.9422233  -0.4229294   0.7558929  -0.66391647
   0.7346618  -0.82061553 -0.21993777 -0.40570587 -1.5468531   0.13747853
   1.0168769  -0.31940016  0.31438437  0.75913024 -1.0571525   0.15604538
   1.9411086  -1.5386285 ]], shape=(3, 20), dtype=float32)


<tf.Tensor: id=1005, shape=(), dtype=float32, numpy=-0.658394>